In [8]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

def get_business_info(api_key, location, keyword):
    # Define the API endpoint
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

    # Parameters for the Places API request
    params = {
        "query": keyword,
        "location": location,
        "radius": 5000,  # Search within a 5km radius of the given location
        "key": api_key,
    }

    # Send the API request
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        return data.get("results", [])
    else:
        print("Error: Failed to fetch data from Google Maps API.")
        return []
    
def categorize_website(url):
    if url == "N/A":
        return "Website Not Available"
    elif "google" in url:
        return "Free google site"
    else:
        return "Own website"
    

def main():
    # Replace "YOUR_API_KEY" with your actual Google Maps API key
    api_key = "#api_key"
    location = "20.5937° N, 78.9629° E" # latitude,longitude

    # Replace "YOUR_KEYWORD" with the business type or name you want to search for
    # For example: "restaurants", "coffee shops", "pharmacies", etc.
    keyword = "garments and clothes shops"

    business_info = get_business_info(api_key, location, keyword)

    # Initialize an empty list for data
    data = []

    # Process the retrieved data and create a DataFrame
    for business in business_info:
        name = business.get("name", "N/A")
        contact_number = business.get("formatted_phone_number", "N/A")
        address = business.get("formatted_address", "N/A")
        website = business.get("website", "N/A")
        data.append([name, contact_number, address, website])
        
    df = pd.DataFrame(data, columns=["Name of business", "Contact Number of business", "Location/address of business", "Website URL"])

    # Replace missing website URLs and contact numbers with "N/A"
    df["Website URL"].fillna("N/A", inplace=True)
    df["Contact Number of business"].fillna("N/A", inplace=True)

    # Create a new column to categorize websites as "Own website," "Free google site," or "Website Not Available"
    df["Website Category"] = df["Website URL"].apply(categorize_website)

    # Prepare a list of businesses and their website categories state-wise
    state_wise_data = df.groupby(["Location/address of business", "Website Category"]).size().unstack(fill_value=0)

    # Create visual charts for every state
    for state, data in state_wise_data.iterrows():
        data.plot(kind="bar", color=["green", "orange", "red"])
        plt.title(f"Website Categories for Businesses in {state}")
        plt.xlabel("Website Category")
        plt.ylabel("Number of Businesses")
        plt.xticks(rotation=45)
        plt.legend(loc="upper right")
        plt.tight_layout()
        plt.show()

if __name__ == "__main__":
    main()    

